# Week 2 Assignment - Code

#### 1. Read all files whose name contains 'list' in the './Week_2/' folder
#### 2. How many lines are invalid?
#### 3. How many lines contain 2 or more molecules (multiple molecules within one smiles string are separated by '.')?

In [1]:
import glob
from rdkit import Chem
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')

filenames = glob.glob('./Week_2/*list*')

All_smiles_list = []
invalid_count = 0
two_or_more = 0

for filename in filenames:
    with open(filename,'r',encoding='utf8') as f:
        for a_line in f:
        #for a_line in open(filename,'r'):
        
            if None in [Chem.MolFromSmiles(smi) for smi in a_line.split('.')]:
                invalid_count += 1
            else:
                if len(a_line.split('.')) > 1:
                    two_or_more += 1
                All_smiles_list += a_line.split('.')

In [2]:
print(invalid_count)
print(two_or_more)
print(len(All_smiles_list))

10
11
7184


#### 4. How many molecules are left after removing duplicates?
#### 5. What was the 'non-canonical' smiles of the canonical smiles O=C1CCCCC1?

In [3]:
All_canonical_smiles_list = [Chem.MolToSmiles(Chem.MolFromSmiles(smi)) for smi in All_smiles_list]
for i in range(len(All_smiles_list)):
    if All_canonical_smiles_list[i] == 'O=C1CCCCC1':
        print(All_smiles_list[i])

All_canonical_smiles_list = list(set(All_canonical_smiles_list))
print(len(All_canonical_smiles_list))

C1CCC(=O)CC1

O=C1CCCCC1

C1CCC(=O)CC1

6825


(Start from the list of canonicalized SMILES)

#### 6. How many smiles strings contain stereochemistry information?

Two ways: (1) If the string contains '@' or '/' or '\\', it contains stereochemistry information

(2) MolFromSmiles -> MolToInchiKey -> stereoisomers do not contain 'UHFFFAOYSA'

In [4]:
mol_objects_all = [Chem.MolFromSmiles(smi) for smi in All_canonical_smiles_list]
inchikeys_all = [ Chem.MolToInchiKey(mol) for mol in mol_objects_all]

print(sum([True for smi in All_canonical_smiles_list if len(set(smi) & set('/@\\'))]))
print(sum([True for ikey in inchikeys_all if 'UHFFFAOYSA' not in ikey]))

31
31


#### 7. How many molecules have Cl?

Write code in two ways: (1) 'Cl' in smiles string,  (2) using GetAtoms() and atom.GetSymbol() in RDKit

#### 8. How many molecules have more than three oxygen atoms?

In [5]:
Clcount_1 = sum([True for smi in All_canonical_smiles_list if 'Cl' in smi])

Clcount_2 = 0
more_than_3_O_count = 0
for mol in mol_objects_all:
    symbol_list = [ atom.GetSymbol() for atom in mol.GetAtoms()]
    if 'Cl' in symbol_list:
        Clcount_2 +=1
    
    if symbol_list.count('O') > 3:
        more_than_3_O_count += 1
        
print(Clcount_1)
print(Clcount_2)
print(more_than_3_O_count)

16
16
465


#### 9. How many molecules are neutral/cation/anion/radicals?
#### 10. How many molecules have molecular weight > 100?

In [6]:
from rdkit.Chem.Descriptors import MolWt, NumRadicalElectrons
from rdkit.Chem.rdmolops import GetFormalCharge

MolWt_100_count = 0

for smi in All_canonical_smiles_list:
    mol = Chem.MolFromSmiles(smi)
    if MolWt(mol) > 100.0:
        MolWt_100_count += 1
        
print(MolWt_100_count)

6302


In [7]:
# --OR --

print(sum([True for smi in All_canonical_smiles_list if MolWt(Chem.MolFromSmiles(smi)) > 100.0]))
print(sum([True for smi in All_canonical_smiles_list if NumRadicalElectrons(Chem.MolFromSmiles(smi)) > 0]))
print(sum([True for smi in All_canonical_smiles_list if GetFormalCharge(Chem.MolFromSmiles(smi)) > 0]))
print(sum([True for smi in All_canonical_smiles_list if GetFormalCharge(Chem.MolFromSmiles(smi)) < 0]))
print(sum([True for smi in All_canonical_smiles_list if GetFormalCharge(Chem.MolFromSmiles(smi)) == 0]))

6302
3
0
6
6819


In [8]:
f = open('./Week_3/smi_list_from_Week_2','w')
for smi in All_canonical_smiles_list:
    f.write(smi+'\n')
f.close()